In [1]:
import os 
import DataFunctions as utils
import Visualisation as vis
from toolkit import * 
import numpy as np
import pandas as pd
### Load data

import pandas as pd
import pickle

# import GDSC2 drug response data using pickle

with open('data/drug-response/GDSC2/cache_gdsc2.pkl', 'rb') as f:
    gdsc2 = pickle.load(f)
    gdsc2_info = pickle.load(f)

# import CCLE gene expression data using pickle

with open('data/gene-expression/CCLE_Public_22Q2/ccle_expression.pkl', 'rb') as f:
    gene_entrez = pickle.load(f)
    ccle = pickle.load(f)

# import CCLE sample info data using pickle

with open('data/gene-expression/CCLE_Public_22Q2/ccle_sample_info.pkl', 'rb') as f:
    ccle_sample_info = pickle.load(f)

# import STRING database using pickle

with open('data/protein-interaction/STRING/string_df.pkl', 'rb') as f:
    string_df = pickle.load(f)
    string_df_info = pickle.load(f)
    string_df_alias = pickle.load(f)


# import proteomic expression
with open('data/proteomic-expression/goncalves-2022-cell/goncalve_proteome_processed.pkl', 'rb') as f:
    joined_full_protein_matrix = pickle.load(f)
    joined_sin_peptile_exclusion_matrix = pickle.load(f)

# import STRING database using pickle

with open('data/protein-interaction/STRING/string_df.pkl', 'rb') as f:
    string_df = pickle.load(f)
    string_df_info = pickle.load(f)
    string_df_alias = pickle.load(f)

# open STRING to goncalves mapping file

with open('data\protein-interaction\STRING\goncalve_to_string_id_df.pkl', 'rb') as f:
    goncalve_to_string_id_df = pickle.load(f)

# open the cache for neighbourhood calculations

with open('data/protein-interaction/STRING/palbociclib_nth_degree_neighbours.pkl', 'rb') as f:
    nth_degree_neighbours = pickle.load(f)


c:\Github\ode-biomarker-project\dynamic-marker\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# create the full dataset
drug_selected = 'Palbociclib'
target_variable = 'AUC'
imputation_method = 'NA' # or "Zero" or "NA"

data_df = utils.create_joint_dataset_from_proteome_gdsc(drug_selected, joined_sin_peptile_exclusion_matrix, gdsc2, drug_value=target_variable)
if imputation_method == 'FirstQuantile':
    imputer = FirstQuantileImputer()
    imputer.fit(data_df)
    # transform the data
    data_df = imputer.transform(data_df, return_df=True)
elif imputation_method == 'Zero':
    data_df.fillna(0, inplace=True)

feature_data, label_data = utils.create_feature_and_label(data_df, label_name=target_variable)

# gene_df = utils.create_joint_dataset_from_ccle_gdsc2(drug_selected, gdsc2, ccle, ccle_sample_info)
# gene_feature_data, gene_label_data = utils.create_feature_and_label(gene_df, label_name=target_variable)


In [3]:
data_df.to_csv('palbociclib_data.csv', index=False)

### Model ID matched with UniProt Columns

In [4]:
joined_sin_peptile_exclusion_matrix.head()

,P37108;SRP14_HUMAN,Q96JP5;ZFP91_HUMAN,Q9Y4H2;IRS2_HUMAN,P36578;RL4_HUMAN,Q6SPF0;SAMD1_HUMAN,O76031;CLPX_HUMAN,Q8WUQ7;CATIN_HUMAN,A6NIH7;U119B_HUMAN,Q9BTD8;RBM42_HUMAN,Q9P258;RCC2_HUMAN,...,P33151;CADH5_HUMAN,Q5EBL4;RIPL1_HUMAN,P49715;CEBPA_HUMAN,Q5TA45;INT11_HUMAN,O14924;RGS12_HUMAN,Q7Z3B1;NEGR1_HUMAN,O60669;MOT2_HUMAN,Q13571;LAPM5_HUMAN,Q96JM2;ZN462_HUMAN,P35558;PCKGC_HUMAN
model_id,,,,,,,,,,,,,,,,,,,,,
SIDM00018,7.10955,3.41494,NaN,7.86661,3.89547,4.19666,NaN,NaN,3.19088,7.35806,...,NaN,NaN,3.90064,2.63998,NaN,NaN,NaN,NaN,NaN,NaN
SIDM00023,6.82802,4.14346,2.23781,7.62878,3.19811,4.60902,NaN,2.47059,3.69535,5.70790,...,NaN,NaN,NaN,3.19608,NaN,NaN,NaN,NaN,NaN,NaN
SIDM00040,7.01426,4.19987,2.44055,8.12459,NaN,4.76881,NaN,NaN,NaN,5.52283,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SIDM00041,5.28591,3.35789,NaN,7.97268,NaN,4.52092,NaN,NaN,2.73088,4.29429,...,NaN,NaN,NaN,2.79023,NaN,NaN,NaN,NaN,NaN,NaN
SIDM00042,5.70786,NaN,NaN,6.22574,NaN,4.49579,NaN,NaN,2.87981,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
ccle_sample_info.head()

,DepMap_ID,cell_line_name,stripped_cell_line_name,CCLE_Name,alias,COSMICID,sex,source,RRID,WTSI_Master_Cell_ID,...,lineage_sub_subtype,lineage_molecular_subtype,default_growth_pattern,model_manipulation,model_manipulation_details,patient_id,parent_depmap_id,Cellosaurus_NCIt_disease,Cellosaurus_NCIt_id,Cellosaurus_issues
0,ACH-000016,SLR 21,SLR21,SLR21_KIDNEY,NaN,NaN,NaN,Academic lab,CVCL_V607,NaN,...,NaN,NaN,NaN,NaN,NaN,PT-JnARLB,NaN,Clear cell renal cell carcinoma,C4033,NaN
1,ACH-000032,MHH-CALL-3,MHHCALL3,MHHCALL3_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,NaN,NaN,Female,DSMZ,CVCL_0089,NaN,...,b_cell,NaN,NaN,NaN,NaN,PT-p2KOyI,NaN,Childhood B acute lymphoblastic leukemia,C9140,NaN
2,ACH-000033,NCI-H1819,NCIH1819,NCIH1819_LUNG,NaN,NaN,Female,Academic lab,CVCL_1497,NaN,...,NSCLC_adenocarcinoma,NaN,NaN,NaN,NaN,PT-9p1WQv,NaN,Lung adenocarcinoma,C3512,NaN
3,ACH-000043,Hs 895.T,HS895T,HS895T_FIBROBLAST,NaN,NaN,Female,ATCC,CVCL_0993,NaN,...,NaN,NaN,2D: adherent,NaN,NaN,PT-rTUVZQ,NaN,Melanoma,C3224,NaN
4,ACH-000049,HEK TE,HEKTE,HEKTE_KIDNEY,NaN,NaN,NaN,Academic lab,CVCL_WS59,NaN,...,NaN,NaN,NaN,immortalized,NaN,PT-qWYYgr,NaN,NaN,NaN,No information is available about this cell li...


In [6]:
gdsc2.head()

,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,TCGA_DESC,DRUG_ID,DRUG_NAME,PUTATIVE_TARGET,PATHWAY_NAME,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE
0,GDSC2,282,13320532,749709,HCC1954,SIDM00872,BRCA,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000098,0.1,-0.251083,0.975203,0.112373,0.906631
1,GDSC2,282,13320565,749710,HCC1143,SIDM00866,BRCA,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000098,0.1,1.343315,0.978464,0.067577,1.683567
2,GDSC2,282,13320598,749711,HCC1187,SIDM00885,BRCA,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000098,0.1,1.736985,0.995100,0.045205,1.875399
3,GDSC2,282,13320631,749712,HCC1395,SIDM00884,BRCA,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000098,0.1,-2.309078,0.867832,0.107282,-0.096212
4,GDSC2,282,13320668,749713,HCC1599,SIDM00877,BRCA,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000098,0.1,-3.106684,0.777532,0.098286,-0.484878


In [7]:
sanger_depmap_map = ccle_sample_info[['DepMap_ID', 'Sanger_Model_ID']]

# drop rows with missing values

sanger_depmap_map = sanger_depmap_map.dropna()

sanger_depmap_map.head()

,DepMap_ID,Sanger_Model_ID
44,ACH-000288,SIDM00122
62,ACH-000467,SIDM00597
94,ACH-000837,SIDM00117
364,ACH-002392,SIDM00030
400,ACH-000001,SIDM00105


In [8]:
sanger_depmap_map.shape

(1153, 2)

In [9]:
# for joined_sin_peptile_exclusion_matrix, only keep the uniprot id but not protein name 


uniprot_only = joined_sin_peptile_exclusion_matrix.copy()

uniprot_columns = uniprot_only.columns.str.split(';').str[0]

uniprot_only.columns = uniprot_columns

In [10]:
uniprot_only.head()

,P37108,Q96JP5,Q9Y4H2,P36578,Q6SPF0,O76031,Q8WUQ7,A6NIH7,Q9BTD8,Q9P258,...,P33151,Q5EBL4,P49715,Q5TA45,O14924,Q7Z3B1,O60669,Q13571,Q96JM2,P35558
model_id,,,,,,,,,,,,,,,,,,,,,
SIDM00018,7.10955,3.41494,NaN,7.86661,3.89547,4.19666,NaN,NaN,3.19088,7.35806,...,NaN,NaN,3.90064,2.63998,NaN,NaN,NaN,NaN,NaN,NaN
SIDM00023,6.82802,4.14346,2.23781,7.62878,3.19811,4.60902,NaN,2.47059,3.69535,5.70790,...,NaN,NaN,NaN,3.19608,NaN,NaN,NaN,NaN,NaN,NaN
SIDM00040,7.01426,4.19987,2.44055,8.12459,NaN,4.76881,NaN,NaN,NaN,5.52283,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SIDM00041,5.28591,3.35789,NaN,7.97268,NaN,4.52092,NaN,NaN,2.73088,4.29429,...,NaN,NaN,NaN,2.79023,NaN,NaN,NaN,NaN,NaN,NaN
SIDM00042,5.70786,NaN,NaN,6.22574,NaN,4.49579,NaN,NaN,2.87981,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# rename index to Sanger_Model_ID

uniprot_only.index.names = ['Sanger_Model_ID']

In [12]:
uniprot_only.head()

,P37108,Q96JP5,Q9Y4H2,P36578,Q6SPF0,O76031,Q8WUQ7,A6NIH7,Q9BTD8,Q9P258,...,P33151,Q5EBL4,P49715,Q5TA45,O14924,Q7Z3B1,O60669,Q13571,Q96JM2,P35558
Sanger_Model_ID,,,,,,,,,,,,,,,,,,,,,
SIDM00018,7.10955,3.41494,NaN,7.86661,3.89547,4.19666,NaN,NaN,3.19088,7.35806,...,NaN,NaN,3.90064,2.63998,NaN,NaN,NaN,NaN,NaN,NaN
SIDM00023,6.82802,4.14346,2.23781,7.62878,3.19811,4.60902,NaN,2.47059,3.69535,5.70790,...,NaN,NaN,NaN,3.19608,NaN,NaN,NaN,NaN,NaN,NaN
SIDM00040,7.01426,4.19987,2.44055,8.12459,NaN,4.76881,NaN,NaN,NaN,5.52283,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SIDM00041,5.28591,3.35789,NaN,7.97268,NaN,4.52092,NaN,NaN,2.73088,4.29429,...,NaN,NaN,NaN,2.79023,NaN,NaN,NaN,NaN,NaN,NaN
SIDM00042,5.70786,NaN,NaN,6.22574,NaN,4.49579,NaN,NaN,2.87981,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
sanger_depmap_map_index = sanger_depmap_map.copy()

sanger_depmap_map_index.set_index('Sanger_Model_ID')

,DepMap_ID
Sanger_Model_ID,
SIDM00122,ACH-000288
SIDM00597,ACH-000467
SIDM00117,ACH-000837
SIDM00030,ACH-002392
SIDM00105,ACH-000001
...,...
SIDM00070,ACH-002393
SIDM00068,ACH-002394
SIDM01257,ACH-002395


In [15]:
df = pd.merge(uniprot_only, sanger_depmap_map_index, left_on='Sanger_Model_ID', right_on='Sanger_Model_ID', how='inner')

In [17]:
df.shape

(948, 6694)

In [18]:
df.head()

,Sanger_Model_ID,P37108,Q96JP5,Q9Y4H2,P36578,Q6SPF0,O76031,Q8WUQ7,A6NIH7,Q9BTD8,...,Q5EBL4,P49715,Q5TA45,O14924,Q7Z3B1,O60669,Q13571,Q96JM2,P35558,DepMap_ID
0,SIDM00018,7.10955,3.41494,NaN,7.86661,3.89547,4.19666,NaN,NaN,3.19088,...,NaN,3.90064,2.63998,NaN,NaN,NaN,NaN,NaN,NaN,ACH-000498
1,SIDM00023,6.82802,4.14346,2.23781,7.62878,3.19811,4.60902,NaN,2.47059,3.69535,...,NaN,NaN,3.19608,NaN,NaN,NaN,NaN,NaN,NaN,ACH-002311
2,SIDM00040,7.01426,4.19987,2.44055,8.12459,NaN,4.76881,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ACH-002202
3,SIDM00041,5.28591,3.35789,NaN,7.97268,NaN,4.52092,NaN,NaN,2.73088,...,NaN,NaN,2.79023,NaN,NaN,NaN,NaN,NaN,NaN,ACH-002306
4,SIDM00042,5.70786,NaN,NaN,6.22574,NaN,4.49579,NaN,NaN,2.87981,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ACH-002186


In [19]:
# remove Sanger_Model_ID column from df 

df.drop('Sanger_Model_ID', axis=1, inplace=True)

df.set_index('DepMap_ID', inplace=True)

df.head()

,P37108,Q96JP5,Q9Y4H2,P36578,Q6SPF0,O76031,Q8WUQ7,A6NIH7,Q9BTD8,Q9P258,...,P33151,Q5EBL4,P49715,Q5TA45,O14924,Q7Z3B1,O60669,Q13571,Q96JM2,P35558
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-000498,7.10955,3.41494,NaN,7.86661,3.89547,4.19666,NaN,NaN,3.19088,7.35806,...,NaN,NaN,3.90064,2.63998,NaN,NaN,NaN,NaN,NaN,NaN
ACH-002311,6.82802,4.14346,2.23781,7.62878,3.19811,4.60902,NaN,2.47059,3.69535,5.70790,...,NaN,NaN,NaN,3.19608,NaN,NaN,NaN,NaN,NaN,NaN
ACH-002202,7.01426,4.19987,2.44055,8.12459,NaN,4.76881,NaN,NaN,NaN,5.52283,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACH-002306,5.28591,3.35789,NaN,7.97268,NaN,4.52092,NaN,NaN,2.73088,4.29429,...,NaN,NaN,NaN,2.79023,NaN,NaN,NaN,NaN,NaN,NaN
ACH-002186,5.70786,NaN,NaN,6.22574,NaN,4.49579,NaN,NaN,2.87981,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# rename: replace '-' with '_' in DepMap_ID

df.rename(index=lambda x: x.replace('-', '_'), inplace=True)

In [22]:
df.head()

,P37108,Q96JP5,Q9Y4H2,P36578,Q6SPF0,O76031,Q8WUQ7,A6NIH7,Q9BTD8,Q9P258,...,P33151,Q5EBL4,P49715,Q5TA45,O14924,Q7Z3B1,O60669,Q13571,Q96JM2,P35558
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH_000498,7.10955,3.41494,NaN,7.86661,3.89547,4.19666,NaN,NaN,3.19088,7.35806,...,NaN,NaN,3.90064,2.63998,NaN,NaN,NaN,NaN,NaN,NaN
ACH_002311,6.82802,4.14346,2.23781,7.62878,3.19811,4.60902,NaN,2.47059,3.69535,5.70790,...,NaN,NaN,NaN,3.19608,NaN,NaN,NaN,NaN,NaN,NaN
ACH_002202,7.01426,4.19987,2.44055,8.12459,NaN,4.76881,NaN,NaN,NaN,5.52283,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACH_002306,5.28591,3.35789,NaN,7.97268,NaN,4.52092,NaN,NaN,2.73088,4.29429,...,NaN,NaN,NaN,2.79023,NaN,NaN,NaN,NaN,NaN,NaN
ACH_002186,5.70786,NaN,NaN,6.22574,NaN,4.49579,NaN,NaN,2.87981,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df.reset_index(inplace=True)

In [ ]:
df.set_index('DepMap_ID', inplace=True)

In [26]:
df.to_csv('data/Goncalves2022_Proteomics.csv')